### Generate smi files and fragment them by mmpdb

In [6]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFMCS
import os, pickle
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import subprocess
import numpy as np

save_output = True
regenerate_smi, refragment, reindex, reproperty, regenerate, reread, recollate, recount, reviz = False, False, False, True, False, False, False, False, False
optimization_task_dict = {'Bioactivity': ['EC50_P21453_1', 'IC50_O43614_1', 'Ki_P14416_1'],
                      'Inhibitory-Efficacy': ['M_C_CYP1A2_inhibitor', 'M_C_CYP2C9_inhibitor', 'M_C_CYP3A4_inhibitor'], 
                      'Toxicity': ['T_C_Ames', 'T_C_NR-AhR', 'T_C_NR-ER']}
train_transformation_dict, mmp_transformation_dict = {}, {}
for optiz_type, task_id_list in optimization_task_dict.items():
    for task_id in tqdm(task_id_list):
        print(f"========================== {task_id} ==========================")
        # Create folder if it doesn't exist
        figures_folder = f'./figures/{task_id}/'
        os.makedirs(figures_folder, exist_ok=True)
        if optiz_type == 'Bioactivity':
            train_filename = f'./OLB-AC-main/LBO-AC-Optimization/{optiz_type}/G_OLB-AC_{task_id}/data/{task_id}_train.csv'
            test_filename = f'./OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_{task_id}/data/{task_id}_test.csv'
            train_df = pd.read_csv(train_filename, header=0, names = ["smiles","value","cano_smiles"],usecols=[1,2,3])
            test_df = pd.read_csv(test_filename,header=0,names=["smiles","value","cano_smiles","matched_smiles","matched_cano_smiles","matched_value"],usecols=[1,2,3,4,5,6])
        else:
            total_filename = f'./OLB-AC-main/LBO-AC-Optimization/{optiz_type}/G_ADMET_{task_id}/data/ADMET/Generation/{task_id}_1.csv'
            total_df = pd.read_csv(total_filename)
            train_df = total_df[total_df['dataset_generate'] == 'training']
            train_df = train_df.reset_index(drop=True)
            train_filename = total_filename.replace(".csv", "_train.csv")
            train_df.to_csv(train_filename, index=False)
            test_df = total_df[total_df['dataset_generate'] == 'test']
            test_df = test_df.reset_index(drop=True)
            test_filename = total_filename.replace(".csv", "_test.csv")
            test_df.to_csv(test_filename, index=False)
        train_df['ID'] = np.arange(len(train_df))
        test_df['ID'] = np.arange(len(test_df))
        train_property_df = train_df[['ID', 'value']]
        train_property_file = train_filename.replace('_train.csv', '_train_properties.csv')
        train_property_df.to_csv(train_property_file, sep='\t', index=False)
        test_property_df = test_df[['ID', 'value']]
        matched_ID = [f'{id}M' for id in test_property_df['ID']]
        matched_values = test_df.matched_value.values
        matched_property_df = test_property_df.copy()
        matched_property_df.loc[:, 'ID'] = matched_ID
        matched_property_df.loc[:, 'value'] = matched_values
        test_merged_property_df = pd.concat([test_property_df, matched_property_df], ignore_index=True)
        test_merged_property_file = test_filename.replace('_test.csv', '_test_properties.csv')
        test_merged_property_df.to_csv(test_merged_property_file, sep='\t', index=False)
        train_smiles = train_df.cano_smiles.values
        test_smiles = test_df.cano_smiles.values
        matched_smiles = test_df.matched_cano_smiles.values
        test_merged_smiles = list(test_smiles) + list(matched_smiles)
        test_merged_ids = test_merged_property_df.ID.values
        test_merged_property = test_merged_property_df.value.values
        print(len(test_merged_ids), len(test_merged_smiles), len(test_merged_property))
    
        # Define the output file paths
        train_smi_file = train_filename.replace('_train.csv', '_train.smi')
        test_smi_file = test_filename.replace('_test.csv', '_test.smi')
        
        # Write smiles to .smi files
        if not os.path.exists(train_smi_file) or regenerate_smi:
            with open(train_smi_file, "w") as f_train:
                for idx, (smile, value) in enumerate(zip(train_df["cano_smiles"], train_df["value"])):
                    f_train.write(f"{smile}\t{idx}\t{value}\n")
        if not os.path.exists(test_smi_file) or regenerate_smi:
            with open(test_smi_file, "w") as f_test:
                for idx, smile, value in zip(test_merged_ids, test_merged_smiles, test_merged_property):
                    f_test.write(f"{smile}\t{idx}\t{value}\n")
                    
        # Fragment smiles to .fragdb files
        train_fragment_file = train_smi_file.replace('.smi', '.fragdb')
        if not os.path.exists(train_fragment_file) or refragment:
            command = f"mmpdb fragment {train_smi_file} -o {train_fragment_file}"
            print(command)
            output = subprocess.run(command, shell=True, capture_output=True, text=True)
            if save_output:
                with open(train_smi_file.replace('.smi', '_fragment_output.txt'), 'w') as f:
                    f.write(output.stderr)
        test_fragment_file = test_smi_file.replace('.smi', '.fragdb')
        if not os.path.exists(test_fragment_file) or refragment:
            command = f"mmpdb fragment {test_smi_file} -o {test_fragment_file}"
            print(command)
            output = subprocess.run(command, shell=True, capture_output=True, text=True)
            if save_output:
                with open(test_smi_file.replace('.smi', '_fragment_output.txt'), 'w') as f:
                    f.write(output.stderr)
    
        # Index to .mmpdb files
        train_index_file = train_fragment_file.replace('.fragdb', '.mmpdb')
        if not os.path.exists(train_index_file) or reindex:
            command = f"mmpdb index {train_fragment_file} -o {train_index_file}"
            print(command)
            output = subprocess.run(command, shell=True, capture_output=True, text=True)
            if save_output:
                with open(train_smi_file.replace('.smi', '_index_output.txt'), 'w') as f:
                    f.write(output.stderr)
        test_index_file = test_fragment_file.replace('.fragdb', '.mmpdb')
        if not os.path.exists(test_index_file) or reindex:
            command = f"mmpdb index {test_fragment_file} -o {test_index_file}"
            print(command)
            output = subprocess.run(command, shell=True, capture_output=True, text=True)
            if save_output:
                with open(test_smi_file.replace('.smi', '_index_output.txt'), 'w') as f:
                    f.write(output.stderr)
            
        # Add properties to the mmpdb databases
        if reproperty:
            command = f"mmpdb loadprops -p {train_property_file} {train_index_file}"
            print(command)
            output = subprocess.run(command, shell=True, capture_output=True, text=True)
            print(output.stderr)
            if save_output:
                with open(train_smi_file.replace('.smi', '_property_output.txt'), 'w') as f:
                    f.write(output.stderr)
        if reproperty:
            command = f"mmpdb loadprops -p {test_merged_property_file} {test_index_file}"
            print(command)
            output = subprocess.run(command, shell=True, capture_output=True, text=True)
            print(output.stderr)
            if save_output:
                with open(test_smi_file.replace('.smi', '_property_output.txt'), 'w') as f:
                    f.write(output.stderr)
    
        # Identify possible transforms and save generated molecules
        for matched_id, matched_smi in zip(matched_ID, matched_smiles):
            save_path = os.path.join(os.path.dirname(test_filename), 'mmpdb', f'{matched_id}.txt')
            if not os.path.exists(os.path.dirname(save_path)):
                os.makedirs(os.path.dirname(save_path))
            if not os.path.exists(save_path) or regenerate:
                command = f"mmpdb transform --smiles '{matched_smi}' {train_index_file} --property value"
                print(command)
                output = subprocess.run(command, shell=True, capture_output=True, text=True)
                # print(output)
                # print(stop)
                if save_output:
                    with open(save_path, 'w') as f:
                        f.write(output.stdout)
            save_csv_path = save_path.replace('.txt', '.csv')
            if not os.path.exists(save_csv_path) or reread:
                try:
                    table_df = pd.read_csv(save_path, sep='\t')
                    table_df.to_csv(save_csv_path, index=False)
                except:
                    print(f"Unable to proceed the molecule {matched_id} in task {task_id}")
    
        for matched_id, matched_smi in zip(test_df.ID.values, test_smiles):
            save_path = os.path.join(os.path.dirname(test_filename), 'mmpdb_reverse', f'{matched_id}.txt')
            if not os.path.exists(os.path.dirname(save_path)):
                os.makedirs(os.path.dirname(save_path))
            if not os.path.exists(save_path) or regenerate:
                command = f"mmpdb transform --smiles '{matched_smi}' {train_index_file} --property value"
                print(command)
                output = subprocess.run(command, shell=True, capture_output=True, text=True)
                if save_output:
                    with open(save_path, 'w') as f:
                        f.write(output.stdout)
            save_csv_path = save_path.replace('.txt', '.csv')
            if not os.path.exists(save_csv_path) or reread:
                try:
                    table_df = pd.read_csv(save_path, sep='\t')
                    table_df.to_csv(save_csv_path, index=False)
                except:
                    print(f"Unable to proceed the molecule {matched_id} in task {task_id}")
    
        # Collate optimized molecules from all generated mmps according to mmp prediciton
        optimized_file = os.path.join(os.path.dirname(test_filename), 'mmpdb', 'optimized.csv')
        reverse_optimized_file = os.path.join(os.path.dirname(test_filename), 'mmpdb_reverse', 'reverse_optimized.csv')
        if not os.path.exists(optimized_file) or recollate:
            column_names = ['smiles', 'mmp_predict', 'value_from_smiles', 'value_to_smiles', 'matched_cano_smiles', 'matched_value', 'matched_id']
            optimized_df = pd.DataFrame(columns=column_names)
            for matched_id, matched_smi, matched_value in zip(matched_ID, matched_smiles, matched_values):
                save_path = os.path.join(os.path.dirname(test_filename), 'mmpdb', f'{matched_id}.csv')
                if os.path.exists(save_path):
                    save_df = pd.read_csv(save_path)
                    for i, row in save_df.iterrows():
                        smiles = row.SMILES
                        mmp_predict = row.value_avg
                        value_from_smiles, value_to_smiles = row.value_from_smiles, row.value_to_smiles # transformations
                        new_data = {'smiles': smiles, 'mmp_predict': mmp_predict, 'value_from_smiles': value_from_smiles, 'value_to_smiles': value_to_smiles, 'matched_cano_smiles': matched_smi, 'matched_value': matched_value, 'matched_id': matched_id}
                        if mmp_predict > 0:
                            optimized_df.loc[len(optimized_df)] = new_data
            optimized_df.to_csv(optimized_file, index=False)
        if not os.path.exists(reverse_optimized_file) or recollate:
            column_names = ['smiles', 'mmp_predict', 'value_from_smiles', 'value_to_smiles', 'matched_cano_smiles', 'matched_value', 'matched_id']
            reverse_optimized_df = pd.DataFrame(columns=column_names)
            for matched_id, matched_smi, matched_value in zip(test_df.ID.values, test_smiles, test_df.value.values):
                save_path = os.path.join(os.path.dirname(test_filename), 'mmpdb_reverse', f'{matched_id}.csv')
                if os.path.exists(save_path):
                    save_df = pd.read_csv(save_path)
                    for i, row in save_df.iterrows():
                        smiles = row.SMILES
                        mmp_predict = row.value_avg
                        value_from_smiles, value_to_smiles = row.value_from_smiles, row.value_to_smiles # transformations
                        new_data = {'smiles': smiles, 'mmp_predict': mmp_predict, 'value_from_smiles': value_from_smiles, 'value_to_smiles': value_to_smiles, 'matched_cano_smiles': matched_smi, 'matched_value': matched_value, 'matched_id': matched_id}
                        if mmp_predict < 0:
                            reverse_optimized_df.loc[len(reverse_optimized_df)] = new_data
            reverse_optimized_df.to_csv(reverse_optimized_file, index=False)
    
        # Visualize transformation distributions
        fig_path = os.path.join(os.path.dirname(test_filename), 'transformations.png')
        if not os.path.exists(fig_path) or reviz:
            # Generate transforamtions in test set
            for matched_id, matched_smi in zip(matched_ID, matched_smiles):
                save_path = os.path.join(os.path.dirname(test_filename), 'mmpdb_test', f'{matched_id}.txt')
                if not os.path.exists(os.path.dirname(save_path)):
                    os.makedirs(os.path.dirname(save_path))
                if not os.path.exists(save_path) or regenerate:
                    command = f"mmpdb transform --smiles '{matched_smi}' {test_index_file} --property value"
                    print(command)
                    output = subprocess.run(command, shell=True, capture_output=True, text=True)
                    if save_output:
                        with open(save_path, 'w') as f:
                            f.write(output.stdout)
                save_csv_path = save_path.replace('.txt', '.csv')
                if not os.path.exists(save_csv_path) or reread:
                    try:
                        table_df = pd.read_csv(save_path, sep='\t')
                        table_df.to_csv(save_csv_path, index=False)
                    except:
                        print(f"Unable to proceed the molecule {matched_id} in task {task_id}")
            # Collate test transformations
            generated_file = os.path.join(os.path.dirname(test_filename), 'mmpdb_test', 'generated.csv')
            if not os.path.exists(generated_file) or recollate:
                column_names = ['smiles', 'mmp_predict', 'value_from_smiles', 'value_to_smiles', 'matched_cano_smiles', 'matched_value', 'matched_id']
                generated_df = pd.DataFrame(columns=column_names)
                for matched_id, matched_smi, matched_value in zip(matched_ID, matched_smiles, matched_values):
                    save_path = os.path.join(os.path.dirname(test_filename), 'mmpdb_test', f'{matched_id}.csv')
                    if os.path.exists(save_path):
                        save_df = pd.read_csv(save_path)
                        for i, row in save_df.iterrows():
                            smiles = row.SMILES
                            mmp_predict = row.value_avg
                            value_from_smiles, value_to_smiles = row.value_from_smiles, row.value_to_smiles # transformations
                            new_data = {'smiles': smiles, 'mmp_predict': mmp_predict, 'value_from_smiles': value_from_smiles, 'value_to_smiles': value_to_smiles, 'matched_cano_smiles': matched_smi, 'matched_value': matched_value, 'matched_id': matched_id}
                            generated_df.loc[len(generated_df)] = new_data
                generated_df.to_csv(generated_file, index=False)
            # Visualize transformations between training and test set
            train_count_file = os.path.join(os.path.dirname(test_filename), 'mmpdb', 'train_transformations.pkl')
            test_count_file = os.path.join(os.path.dirname(test_filename), 'mmpdb', 'test_transformations.pkl')
            overlap_count_file = os.path.join(os.path.dirname(test_filename), 'mmpdb', 'overlapped_transformations.pkl')
            if not os.path.exists(train_count_file) or not os.path.exists(test_count_file) or not os.path.exists(overlap_count_file) or recount:
                optimized_df, reverse_optimized_df = pd.read_csv(optimized_file), pd.read_csv(reverse_optimized_file)
                train_transformations_optimized = [f'{before}>>{after}' for before, after in zip(optimized_df.value_from_smiles.values, optimized_df.value_to_smiles.values)]
                train_transformations_reverse_optimized = [f'{before}>>{after}' for before, after in zip(reverse_optimized_df.value_from_smiles.values, reverse_optimized_df.value_to_smiles.values)]
                train_transformations = train_transformations_optimized + train_transformations_reverse_optimized
                generated_df = pd.read_csv(generated_file)
                test_transformations = [f'{before}>>{after}' for before, after in zip(generated_df.value_from_smiles.values, generated_df.value_to_smiles.values)]
                # Calculate counts for each transformation
                all_transformations = set(train_transformations).union(test_transformations)
                train_count = {trans: train_transformations.count(trans) for trans in all_transformations}
                test_count = {trans: test_transformations.count(trans) for trans in all_transformations}
                overlapping_counts = {trans: min(train_count[trans], test_count[trans]) for trans in all_transformations}
                with open(train_count_file, 'wb') as f:
                    pickle.dump(train_count, f)
                with open(test_count_file, 'wb') as f:
                    pickle.dump(test_count, f)
                with open(overlap_count_file, 'wb') as f:
                    pickle.dump(overlapping_counts, f)
            else:
                with open(train_count_file, 'rb') as f:
                    train_count = pickle.load(f)
                with open(test_count_file, 'rb') as f:
                    test_count = pickle.load(f)
                with open(overlap_count_file, 'rb') as f:
                    overlapping_counts = pickle.load(f)
            
            # Calculate total counts
            # total_train = sum(train_count.values())
            # total_test = sum(test_count.values())
            # total_overlap = sum(overlapping_counts.values())
            total_train, total_test, total_overlap = 0, 0, 0
            for key, value in train_count.items():
                if value > 0:
                    total_train += 1
            for key, value in test_count.items():
                if value > 0:
                    total_test += 1
            for key, value in overlapping_counts.items():
                if value > 0:
                    total_overlap += 1
            
            # Calculate percentages
            train_percentage = total_train / (total_train + total_test + total_overlap) * 100
            test_percentage = total_test / (total_train + total_test + total_overlap) * 100
            overlap_percentage = total_overlap / (total_train + total_test + total_overlap) * 100
            print(f"Task: {task_id}\ntrain_percentage: {train_percentage:.2f}%\ntest_percentage: {test_percentage:.2f}%\noverlap_percentage: {overlap_percentage:.2f}%")
            print(f"------------------------------------------------")

  0%|          | 0/3 [00:00<?, ?it/s]

========================== EC50_P21453_1 ==========================
196 196 196
mmpdb loadprops -p ./OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train_properties.csv ./OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train.mmpdb
Using dataset: MMPs from './OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train.fragdb'
Reading properties from './OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train_properties.csv'
Read 1 properties for 2860 compounds from './OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train_properties.csv'
785 compounds from './OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train_properties.csv' are not in the dataset at './OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train.mmpdb'
Imported 2075 'val

  0%|          | 0/3 [00:00<?, ?it/s]

========================== M_C_CYP1A2_inhibitor ==========================
60 60 60
mmpdb loadprops -p ./OLB-AC-main/LBO-AC-Optimization/Inhibitory-Efficacy/G_ADMET_M_C_CYP1A2_inhibitor/data/ADMET/Generation/M_C_CYP1A2_inhibitor_1_train_properties.csv ./OLB-AC-main/LBO-AC-Optimization/Inhibitory-Efficacy/G_ADMET_M_C_CYP1A2_inhibitor/data/ADMET/Generation/M_C_CYP1A2_inhibitor_1_train.mmpdb
Using dataset: MMPs from './OLB-AC-main/LBO-AC-Optimization/Inhibitory-Efficacy/G_ADMET_M_C_CYP1A2_inhibitor/data/ADMET/Generation/M_C_CYP1A2_inhibitor_1_train.fragdb'
Reading properties from './OLB-AC-main/LBO-AC-Optimization/Inhibitory-Efficacy/G_ADMET_M_C_CYP1A2_inhibitor/data/ADMET/Generation/M_C_CYP1A2_inhibitor_1_train_properties.csv'
Read 1 properties for 11343 compounds from './OLB-AC-main/LBO-AC-Optimization/Inhibitory-Efficacy/G_ADMET_M_C_CYP1A2_inhibitor/data/ADMET/Generation/M_C_CYP1A2_inhibitor_1_train_properties.csv'
4257 compounds from './OLB-AC-main/LBO-AC-Optimization/Inhibitory-Effic

  0%|          | 0/3 [00:00<?, ?it/s]

========================== T_C_Ames ==========================
624 624 624
mmpdb fragment ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_train.smi -o ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_train.fragdb
mmpdb fragment ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_test.smi -o ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_test.fragdb
mmpdb index ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_train.fragdb -o ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_train.mmpdb
mmpdb index ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_test.fragdb -o ./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_T_C_Ames/data/ADMET/Generation/T_C_Ames_1_test.mmpdb
mmpdb loadprops -p ./OL

In [8]:
import os, pickle
from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D, rdDepictor

task_smiles = {'EC50_P21453_1': [['CCCc1ccc(-c2noc3c2CCc2cc(CN4CC(C(=O)O)C4)ccc2-3)cc1',
                                'CCCc1ccc(-c2noc3c2COc2cc(CN4CC(C(=O)O)C4)ccc2-3)cc1'],
                                 ['CCc1c(CCCC(=O)O)cccc1-c1nsc(-c2ccc(OC(C)C)c(C#N)c2)n1',
                                'CCc1c(CCCC(=O)O)cccc1-c1noc(-c2ccc(OC(C)C)c(C#N)c2)n1'],
                                 ['Cc1cc(-c2cnc(-c3sc(C)c4c3CC3C4C3(C)C)s2)cc(C)c1OCC(O)CO',
                                'Cc1cc(-c2cnc(-c3sc(C)c4c3CC3C4C3(C)C)o2)cc(C)c1OCC(O)CO']
                                ],
               'IC50_O43614_1': [['CCOc1ccccc1-c1noc(C2CCCN2C(=O)c2cc(F)ccc2-n2nccn2)n1',
                                'CCOc1ccccc1-c1noc(C2CCCN2C(=O)c2cc(C)ccc2-n2nccn2)n1'],
                                 ['COc1cccc(OC)c1C1CCCC(=O)N1Cc1ccnc(N2CCCC2)c1',
                                'COc1cccc(OC)c1C1CCCC(=O)N1Cc1cccc(N2CCCC2)c1'],
                                 ['CC1COc2ccc(F)cc2C(C)N1C(=O)c1ccc(Cl)cc1',
                                'Cc1ccc2c(c1)C(C)N(C(=O)c1ccc(Cl)cc1)C(C)CO2'],
                                 ['COc1cncc(OC)c1C1CCCC(=O)N1Cc1cccc(-c2csc(C)n2)c1',
                                'COc1cccc(OC)c1C1CCCC(=O)N1Cc1cccc(-c2csc(C)n2)c1'],
                                 ['Cc1ccc(-n2nccn2)c(C(=O)N2CCOCC2Cc2cccc(-n3cncn3)c2)c1',
                                'Cc1ccc(-n2nccn2)c(C(=O)N2CCOCC2Cc2cccc(-n3cccn3)c2)c1'],
                                 ['COc1cncc(OC)c1C1CCCCC(=O)N1Cc1cccc(-c2csc(C)n2)c1',
                                'COc1cccc(OC)c1C1CCCCC(=O)N1Cc1cccc(-c2csc(C)n2)c1'],
                                 ['COc1cccc(OC)c1C1CCCC(=O)N1Cc1cccc(-n2cncn2)c1',
                                'COc1cccc(OC)c1C1CCCC(=O)N1Cc1cccc(-n2cccn2)c1'],
                                 ['COc1ncnc(OC)c1C1CCCC(=O)N1Cc1cccc(-c2csc(C)n2)c1',
                                'COc1ccnc(OC)c1C1CCCC(=O)N1Cc1cccc(-c2csc(C)n2)c1'],
                                 ['CCOc1ncccc1-c1noc(C2CCCN2C(=O)c2cc(F)ccc2-n2nccn2)n1',
                                'CCOc1ncccc1-c1noc(C2CCCN2C(=O)c2cc(C)ccc2-n2nccn2)n1']
                                ], 
               'Ki_P14416_1': [['CCCN1CCC(c2coc3ccccc23)CC1',
                                'CCCN1CCC(c2csc3ccccc23)CC1'],
                                 ['CCCN(CCc1ccc(C)cc1)C1CCc2c(O)cccc2C1',
                                'CCCN(CCc1ccc(N)cc1)C1CCc2c(O)cccc2C1'],
                                 ['Cl.O=C(NCCCCN1CCN(c2cccc(Cl)c2Cl)CC1)c1cc2ccccc2o1',
                                'Cl.O=C(NCCCCN1CCN(c2cccc(Cl)c2Cl)CC1)c1cc2ccccc2s1'],
                                 ['CN1CCN(C2=Nc3ccccc3Oc3ccc(Cl)cc32)CC1',
                                'CN1CCN(C2=Nc3ccccc3Sc3ccc(Cl)cc32)CC1'],
                                 ['O=S(=O)(NCCCCN1CCN(c2noc3ccccc23)CC1)c1cccc2scnc12',
                                'O=S(=O)(NCCCCN1CCN(c2nsc3ccccc23)CC1)c1cccc2scnc12']
                                ],
              'M_C_CYP1A2_inhibitor': [
                  ['O=S(=O)(O)c1nc2ccccc2n1Cc1ccccc1', 'CS(=O)(=O)c1nc2ccccc2n1Cc1ccccc1'],
                  ['COc1cccc(C2=NOC(C(=O)NCc3ccccn3)C2)c1', 'COc1cccc(C2=NOC(C(=O)NCc3ccccc3)C2)c1']
              ],
              'M_C_CYP2C9_inhibitor': [
                  ['O=c1c(-c2ccc(F)cc2)nc2cnc(N3CCOCC3)nc2n1Cc1cccs1', 'O=c1c(-c2ccc(F)cc2)nc2cnc(N3CCNCC3)nc2n1Cc1cccs1'],
                  ['COc1cccc(Cn2c(=O)c(-c3ccc(F)cc3)nc3cnc(N4CCOCC4)nc32)c1', 'COc1cccc(Cn2c(=O)c(-c3ccc(F)cc3)nc3cnc(N4CCNCC4)nc32)c1']
              ],
              'M_C_CYP3A4_inhibitor': [
                  ['O=C(c1ccncc1)N1CCC2(CCCN(c3ccncc3)C2)CC1', 'O=C(c1ccncc1)N1CCC2(CCCN(c3ccccc3)C2)CC1'],
                  ['COCC(=O)N1CCCC2(CCN(c3ccccn3)C2)C1', 'COCC(=O)N1CCCC2(CCN(c3ccccc3)C2)C1']
              ], 
              'T_C_NR-AhR': [
                  ['Nc1ccc(-c2ccccc2)cc1', 'Oc1ccc(-c2ccccc2)cc1'],
                  ['COc1ccc(N)cc1', 'COc1ccc(C)cc1'],
                  ['Cc1ccc([N+](=O)[O-])cc1N', 'Cc1ccc([N+](=O)[O-])cc1C'],
                  ['Cc1cccc(O)c1N', 'Cc1cccc(O)c1C'],
                  ['Oc1ccccc1O', 'Cc1ccccc1O']
              ], 
              'T_C_NR-ER': [
                  ['CC(O)CN(C)C', 'CC(C)CC(C)O'],
                  ['Nc1ccc(-c2ccccc2)cc1', 'Cc1ccc(-c2ccccc2)cc1'],
                  ['Cc1ccc(N(C)C)cc1', 'Cc1ccc(C(C)C)cc1'],
                  ['C[C@@H](O)C(=O)O', 'C[C@@H](N)C(=O)O'],
                  ['OCCCCCO', 'OCCNCCO']
              ], 
              'T_C_Ames': [
                  ['Nc1ccccc1-c1ccccc1', 'Cc1ccccc1-c1ccccc1'],
                  ['Cc1ccccc1N', 'Cc1ccccc1C'],
                  ['OCCCCl', 'CCCCCl'],
                  ['CCCCOO', 'CCCCCO'],
                  ['Nc1ccc2c(c1)C(=O)c1ccccc1C2=O', 'Cc1ccc2c(c1)C(=O)c1ccccc1C2=O'],
                  ['Cc1ccc(N)c(O)c1', 'Cc1ccc(C)c(O)c1'],
                  ['CCl', 'CCS'],
                  ['C=CCNC(N)=O', 'C=CCNC(N)=S'],
                  ['Nc1cccc(N)c1', 'Cc1cccc(N)c1'],
                  ['Cc1cc(C)cc(N)c1', 'Cc1cc(C)cc(C)c1'],
                  ['c1ccc2ncccc2c1', 'c1ccc2ccccc2c1'],
                  ['CN(C)N', 'CN(C)C'],
                  ['Oc1cccc2ncccc12', 'Oc1cccc2ccccc12'],
                  ['Nc1cc(Cl)ccc1O', 'Cc1cc(Cl)ccc1O'],
                  ['C=Cc1ccc(N)cc1', 'C=Cc1ccc(C)cc1'],
                  ['CCOP(=O)(OCC)Oc1ccc([N+](=O)[O-])cc1', 'CCOP(=S)(OCC)Oc1ccc([N+](=O)[O-])cc1']
              ]
              }

for task_id, mmps in task_smiles.items():
    print(f"========================== {task_id} ==========================")
    task_novel_counter = 0
    if task_id in ['EC50_P21453_1', 'IC50_O43614_1', 'Ki_P14416_1']:
        test_filename = f'./OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_{task_id}/data/{task_id}_test.csv'
    elif task_id in ['M_C_CYP1A2_inhibitor', 'M_C_CYP2C9_inhibitor', 'M_C_CYP3A4_inhibitor']:
        test_filename = f'./OLB-AC-main/LBO-AC-Optimization/Inhibitory-Efficacy/G_ADMET_{task_id}/data/ADMET/Generation/{task_id}_1_test.csv'
    else:
        test_filename = f'./OLB-AC-main/LBO-AC-Optimization/Toxicity/G_ADMET_{task_id}/data/ADMET/Generation/{task_id}_1_test.csv'
    for idx, mmp in enumerate(mmps):
        # Define the reactant and product SMILES
        reactant_smiles, product_smiles = mmp[0], mmp[1]
        print(reactant_smiles)
        print(product_smiles)
        
        # Convert SMILES to RDKit molecules
        reactant_mol = Chem.MolFromSmiles(reactant_smiles)
        product_mol = Chem.MolFromSmiles(product_smiles)
        
        # Read test transformations
        test_count_file = os.path.join(os.path.dirname(test_filename), 'mmpdb', 'test_transformations.pkl')
        with open(test_count_file, 'rb') as f:
            test_count = pickle.load(f)
            
        # Define a list of reaction SMARTS strings or templates
        reaction_smarts = [key for key in test_count.keys()]
        
        # Convert SMARTS strings to reaction objects
        reactions = [AllChem.ReactionFromSmarts(smarts) for smarts in reaction_smarts]
        
        # Check each reaction to see if it transforms the reactant into the product
        for i, reaction in enumerate(reactions):
            products = reaction.RunReactants((reactant_mol,))
            if products:
                product = products[0][0]
                if Chem.MolToSmiles(product) == product_smiles:
                    print("Found a plausible reaction:")
                    reaction_smart = reaction_smarts[i]
                    print(reaction_smart)
                    # Read train transformations
                    train_count_file = os.path.join(os.path.dirname(test_filename), 'mmpdb', 'train_transformations.pkl')
                    with open(train_count_file, 'rb') as f:
                        train_count = pickle.load(f)
                    if train_count[reaction_smart] > 0:
                        print("This reaction can be found in the training set")
                    else:
                        task_novel_counter += 1
                        print("This reaction cannot be found in the training set")
                        # Draw the reaction
                        # Compute the invariant part (core structure)
                        reactant = reaction.GetReactantTemplate(0)
                        drawer = rdMolDraw2D.MolDraw2DSVG(-1,-1)
                        dopts = drawer.drawOptions()
                        Draw.SetACS1996Mode(dopts, 0.70)
                        dopts.bondLineWidth = 1.5 # default is 2.0
                        drawer.DrawReaction(reaction)
                        drawer.FinishDrawing()
                        
                        # Save the reaction figure
                        reaction_figure_path = f'./results/optimization/transforamtion/{task_id}/{idx}_{i}_novel_reaction.svg'
                        if not os.path.exists(os.path.dirname(reaction_figure_path)):
                            os.makedirs(os.path.dirname(reaction_figure_path))         
                        with open(reaction_figure_path, 'w') as f:
                            f.write(drawer.GetDrawingText())
                            
                        # Draw common part
                        common_part = Chem.MolFromSmarts(Chem.rdFMCS.FindMCS([reactant_mol, product_mol], completeRingsOnly=False).smartsString)
                        drawer = rdMolDraw2D.MolDraw2DSVG(-1,-1)
                        dopts = drawer.drawOptions()
                        Draw.SetACS1996Mode(dopts, 0.70)
                        dopts.bondLineWidth = 1.5 # default is 2.0
                        drawer.DrawMolecule(common_part)
                        drawer.FinishDrawing()
                        common_part_file = reaction_figure_path.replace(f'{idx}_{i}_novel_reaction.svg', f'{idx}_{i}_common_part.svg')
                        with open(common_part_file, 'w') as f:
                            f.write(drawer.GetDrawingText())

                        # Draw reactant molecule
                        drawer = rdMolDraw2D.MolDraw2DSVG(-1,-1)
                        dopts = drawer.drawOptions()
                        Draw.SetACS1996Mode(dopts, 0.70)
                        dopts.bondLineWidth = 1.5 # default is 2.0
                        drawer.DrawMolecule(reactant_mol)
                        drawer.FinishDrawing()
                        reactant_file = reaction_figure_path.replace(f'{idx}_{i}_novel_reaction.svg', f'{idx}_{i}_reactant.svg')
                        with open(reactant_file, 'w') as f:
                            f.write(drawer.GetDrawingText())

                        # Draw product molecule
                        drawer = rdMolDraw2D.MolDraw2DSVG(-1,-1)
                        dopts = drawer.drawOptions()
                        Draw.SetACS1996Mode(dopts, 0.70)
                        dopts.bondLineWidth = 1.5 # default is 2.0
                        drawer.DrawMolecule(product_mol)
                        drawer.FinishDrawing()
                        product_file = reaction_figure_path.replace(f'{idx}_{i}_novel_reaction.svg', f'{idx}_{i}_product.svg')
                        with open(product_file, 'w') as f:
                            f.write(drawer.GetDrawingText())
                    break
        else:
            print("No plausible reaction found.")
        print(f"------------------------------------------------")
    print(f'========================== {task_id}: {task_novel_counter} novel transformation are found ==========================')

========================== EC50_P21453_1 ==========================
CCCc1ccc(-c2noc3c2CCc2cc(CN4CC(C(=O)O)C4)ccc2-3)cc1
CCCc1ccc(-c2noc3c2COc2cc(CN4CC(C(=O)O)C4)ccc2-3)cc1
No plausible reaction found.
------------------------------------------------
CCc1c(CCCC(=O)O)cccc1-c1nsc(-c2ccc(OC(C)C)c(C#N)c2)n1
CCc1c(CCCC(=O)O)cccc1-c1noc(-c2ccc(OC(C)C)c(C#N)c2)n1
Found a plausible reaction:
[*:1]c1nsc([*:2])n1>>[*:1]c1noc([*:2])n1
This reaction cannot be found in the training set
------------------------------------------------
Cc1cc(-c2cnc(-c3sc(C)c4c3CC3C4C3(C)C)s2)cc(C)c1OCC(O)CO
Cc1cc(-c2cnc(-c3sc(C)c4c3CC3C4C3(C)C)o2)cc(C)c1OCC(O)CO
No plausible reaction found.
------------------------------------------------
========================== EC50_P21453_1: 1 novel transformation are found ==========================
========================== IC50_O43614_1 ==========================
CCOc1ccccc1-c1noc(C2CCCN2C(=O)c2cc(F)ccc2-n2nccn2)n1
CCOc1ccccc1-c1noc(C2CCCN2C(=O)c2cc(C)ccc2-n2nccn2)n1
Found a 

In [3]:
!mmpdb list --all ./OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train.mmpdb

                                                Name                                                #cmpds #rules #pairs #envs  #stats  |--------------------------------------------------- Title ----------------------------------------------------| Properties
./OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train.mmpdb   2075  40318 405510 259236 259236  MMPs from './OLB-AC-main/LBO-AC-Optimization/Bioactivity/G_OLB-AC_EC50_P21453_1/data/EC50_P21453_1_train.fragdb' value
      Created: 2024-05-09 15:55:52.457450
        #compounds/property:  2075/value
        #smiles for rules: 4891  for constants: 3570
        Fragment options:
          cut_smarts: [#6+0;!$(*=,#[!#6])]!@!=!#[!#0;!#1;!$([CH2]);!$([CH3][CH2])]
          max_heavies: 100
          max_rotatable_bonds: 10
          method: chiral
          num_cuts: 3
          rotatable_smarts: [!$([NH]!@C(=O))&!D1&!$(*#*)]-&!@[!$([NH]!@C(=O))&!D1&!$(*#*)]
          salt_remover: <default>
       

In [4]:
!mmpdb list --help

usage: mmpdb list [-h] [--all] [--quiet] [--recount] [DATABASE ...]

positional arguments:
  DATABASE     zero or more database locations (eg, 'csd.mmpdb')

optional arguments:
  -h, --help   show this help message and exit
  --all, -a    list all information about the dataset
  --quiet, -q  do not show progress or status information
  --recount    count the table sizes directly, instead of using cached data

In the simplest case, look in the current directory for files matching
'*.mmpdb', open each one, and print a terse summary of the information
in the database.

  % mmpdb list
               Name             #cmpds  #rules  #pairs   #envs    #stats   |-------- Title ---------| Properties
  CHEMBL_thrombin_Ki_IC50.mmpdb   2985   29513   258294   199631        0  thrombin Ki IC50           
                      csd.mmpdb  16843 2525166 15328608 15199376 15199376  MMPs from 'csd.fragments'  MP

The output is a set of columns. The first line is the header. The first
column contains th

In [5]:
!mmpdb help-analysis


The overall process is:

  1) Fragment structures in a SMILES file, to produce fragments.

  2) Index the fragments to produces matched molecular pairs.
     (you might include property information at this point)

  3) Load property information.

  4) Find transforms for a given structure; and/or

  5) Predict a property for a structure given the known
     property for another structure

Some terminology:
A fragmentation cuts 1, 2, or 3 non-ring bonds to
convert a structure into a "constant" part and a "variable" part. The
substructure in the variable part is a single fragment, and often
considered the R-groups, while the constant part contains one
fragment for each cut, and it often considered as containing the
core.

The matched molecular pair indexing process finds all pairs which have
the same constant part, in order to define a transformation from one
variable part to another variable part. A "rule" stores information
about a transformation, including a list of all the pairs for